<h1>Sommaire<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-data" data-toc-modified-id="Import-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import data</a></span><ul class="toc-item"><li><span><a href="#Train-set" data-toc-modified-id="Train-set-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Train set</a></span></li><li><span><a href="#Val-set" data-toc-modified-id="Val-set-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Val set</a></span></li></ul></li><li><span><a href="#Prepare-data" data-toc-modified-id="Prepare-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare data</a></span><ul class="toc-item"><li><span><a href="#Bag-of-words-post" data-toc-modified-id="Bag-of-words-post-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Bag of words post</a></span></li><li><span><a href="#Binarize-tags" data-toc-modified-id="Binarize-tags-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Binarize tags</a></span></li></ul></li><li><span><a href="#Logistic-regression-multilabels-(one-vs-rest)" data-toc-modified-id="Logistic-regression-multilabels-(one-vs-rest)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Logistic regression multilabels (one vs rest)</a></span></li><li><span><a href="#Evaluation-of-Logistic-regression-multilabels-(one-vs-rest)" data-toc-modified-id="Evaluation-of-Logistic-regression-multilabels-(one-vs-rest)-3"><span class="toc-item-num">4&nbsp;&nbsp;</span>Evaluation of Logistic regression multilabels (one vs rest)</a></span></li><li><span><a href="#SVM-multilabels-(one-vs-rest)" data-toc-modified-id="SVM-multilabels-(one-vs-rest)-4"><span class="toc-item-num">5&nbsp;&nbsp;</span>SVM multilabels (one vs rest)</a></span></li><li><span><a href="#Evaluation-of-SVM-multilabels-(one-vs-rest)" data-toc-modified-id="Evaluation-of-SVM-multilabels-(one-vs-rest)-4"><span class="toc-item-num">6&nbsp;&nbsp;</span>Evaluation of SVM multilabels (one vs rest)</a></span></li><li><span><a href="#Random-forest-multilabels-(one-vs-rest)" data-toc-modified-id="Random-forest-multilabels-(one-vs-rest)-5"><span class="toc-item-num">7&nbsp;&nbsp;</span>Random forest multilabels (one vs rest)</a></span></li><li><span><a href="#Evaluation-of-Random-forest-multilabels-(one-vs-rest)" data-toc-modified-id="Evaluation-of-Random-forest-multilabels-(one-vs-rest)-5"><span class="toc-item-num">8&nbsp;&nbsp;</span>Evaluation of Random forest multilabels (one vs rest)</a></span></li></ul></div>

In [1]:
import os
import pandas as pd
import glob
import numpy as np
import pickle
import gensim
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import GridSearchCV

# Import data

## Train set

In [ ]:
# post
df_X_train = pd.read_pickle('train/X_train_filtre.pkl')
df_X_train.head()

In [3]:
# on garde le corpus 'text_small'
df_X_train = df_X_train.drop(columns=['Text', 'text_large'])
df_X_train.head()

,text_small
195918,"[store, part, separate, page, whenever, prepar..."
495592,"[tool, php, web, developer, live, without, lot..."
451531,"[long, string, newline, tell, compiler, string..."
542821,"[character, pointers, consider, point, area, m..."
549984,"[install, without, desktop, ubuntu, support, p..."


In [4]:
df_X_train.shape

(103225, 1)

In [5]:
# tags
df_Y_train = pd.read_pickle('train/Y_train_filtre.pkl')
df_Y_train.head()

,Tags,tags_filtered
195918,"[html, database, forms, cookies]","[html, database, forms]"
495592,"[php, development-environment]",[php]
451531,"[c++, string]","[c++, string]"
542821,"[c, arrays, pointers]","[c, arrays]"
549984,"[python, librsvg, rsvg]",[python]


In [6]:
# on garde le corpus 'tags_filtered'
df_Y_train = df_Y_train.drop(columns=['Tags'])
df_Y_train.head()

,tags_filtered
195918,"[html, database, forms]"
495592,[php]
451531,"[c++, string]"
542821,"[c, arrays]"
549984,[python]


In [7]:
df_Y_train.shape

(103225, 1)

## Val set

In [8]:
# post
df_X_val = pd.read_pickle('val/X_val_filtre.pkl')
df_X_val.head()

,Text
230075,"[type, java, cache, use, case, data, change, f..."
295941,"[php, show, n, x, b, r, show, character, webpage]"
279171,"[saving, hebrew, utf, database, php, saving, h..."
201268,"[nlog, store, current, thread, user, use, nlog..."
403510,"[add, cgpoint, nsmutablearray, want, store, cg..."


In [9]:
# tags
df_Y_val = pd.read_pickle('val/Y_val_filtre.pkl')
df_Y_val.head()

,Tags,tags_filtered
230075,"[java, performance, multithreading, caching, lru]","[caching, java, multithreading, performance]"
295941,"[php, hidden-characters]",[php]
279171,"[php, mysql, utf-8]","[php, mysql]"
201268,"[c#, .net, nlog]","[.net, c#]"
403510,"[iphone, objective-c, nsarray, cgpoint]","[objective-c, iphone]"


In [10]:
# on garde le corpus 'tags_filtered'
df_Y_val = df_Y_val.drop(columns=['Tags'])
df_Y_val.head()

,tags_filtered
230075,"[caching, java, multithreading, performance]"
295941,[php]
279171,"[php, mysql]"
201268,"[.net, c#]"
403510,"[objective-c, iphone]"


# Prepare data

## Bag of words post

In [11]:
# no parameters : keep all words
vectorizer = CountVectorizer(encoding='unicode')

# train set
corpus = []
for post in df_X_train['text_small']:
    post_text = ' '.join(post)
    corpus.append(post_text)

X_train = vectorizer.fit_transform(corpus)
print('X_train shape : {}'.format(X_train.shape))

# sample train set
# sampling
num_samples = 15000
df_X_train_s = df_X_train.sample(n=num_samples,random_state=1)
index_to_keep = df_X_train_s.index.tolist()
df_Y_train_s = df_Y_train.loc[index_to_keep]

# no parameters : keep all words
vectorizer2 = CountVectorizer(encoding='unicode')

corpus = []
for post in df_X_train_s['text_small']:
    post_text = ' '.join(post)
    corpus.append(post_text)

X_train_s = vectorizer2.fit_transform(corpus)
print('X_train_s shape : {}'.format(X_train_s.shape))

X_train shape : (103225, 4167)
X_train_s shape : (15000, 4166)


In [12]:
# val set
corpus = []
for post in df_X_val['Text']:
    post_text = ' '.join(post)
    corpus.append(post_text)

X_val = vectorizer.transform(corpus)  # no fit, just transform
print('X_val shape : {}'.format(X_val.shape))

X_val shape : (34477, 4167)


## Binarize tags

In [13]:
mlb = MultiLabelBinarizer()

# train set
Y_train = mlb.fit_transform(df_Y_train['tags_filtered'])
tags = mlb.classes_
print('Y_train shape : {}'.format(Y_train.shape))

Y_train shape : (103225, 100)


In [14]:
# val set
Y_val = mlb.transform(df_Y_val['tags_filtered'])
print('Y_val shape : {}'.format(Y_val.shape))

Y_val shape : (34477, 100)


# Logistic regression multilabels (one vs rest)

In [15]:
def evaluate_clf(clf, csv_path=None, clf_name=None):
    # prediction
    # train
    Y_pred_train = clf.predict(X_train)
    # val
    Y_pred_val = clf.predict(X_val)
    print('Nombre de tags présents : {}'.format(sum(sum(Y_val))))
    print('Nombre de tags prédits : {}'.format(sum(sum(Y_pred_val))))
    print('---------------------------------------------------------')
    print(classification_report(Y_val, Y_pred_val,
                                target_names=tags, zero_division=0))
    if csv_path and clf_name:
        train_score_macro = f1_score(Y_train, Y_pred_train, average='macro')
        val_score_macro = f1_score(Y_val, Y_pred_val, average='macro')
        train_score_micro = f1_score(Y_train, Y_pred_train, average='micro')
        val_score_micro = f1_score(Y_val, Y_pred_val, average='micro')
        train_score_weighted = f1_score(Y_train, Y_pred_train, average='weighted')
        val_score_weighted = f1_score(Y_val, Y_pred_val, average='weighted')
        train_score_samples = f1_score(Y_train, Y_pred_train, average='samples')
        val_score_samples = f1_score(Y_val, Y_pred_val, average='samples')
        with open(csv_path, 'w') as file:
            file.write('{};{};{};{};{};{};{};{};{}'.format(clf_name,
                                                           train_score_macro,
                                                           val_score_macro,
                                                           train_score_micro,
                                                           val_score_micro,
                                                           train_score_weighted,
                                                           val_score_weighted,
                                                           train_score_samples,
                                                           val_score_samples))

Afin de determiner les meilleurs paramètres du modèle, nous allons utiliser `GridSearchCV` du module `model_selection` de `scikit-learn`. Nous appliquerons cela à un échantillon de 15000 posts afin de reduire le temps d'exécution.

In [16]:
# classifier
log_clf = OneVsRestClassifier(LogisticRegression(random_state=0,
                                                 max_iter=10000,
                                                 verbose=0))

# hyperparameters
param_grid = [{'estimator__C': [0.1, 0.5, 1., 5, 10]}]

# cross-validation
clf = GridSearchCV(log_clf,
                   param_grid,
                   scoring='f1_macro',
                   n_jobs=5,
                   verbose=0,
                   cv=5,
                   return_train_score=True)


mlb2 = MultiLabelBinarizer()

# sample train set
Y_train_s = mlb2.fit_transform(df_Y_train_s['tags_filtered'])

clf.fit(X_train_s, Y_train_s)

print('Meilleurs hyper-paramètres :\n')
print(clf.best_params_)

Meilleurs hyper-paramètres :

{'estimator__C': 10}


Une fois le meilleur paramètre determiner, nous appliquerons la classification à toute la base en considérant le meilleur paramètre déterminé précédemment.

In [17]:
# train classifier
clf = OneVsRestClassifier(LogisticRegression(random_state=0,
                                                 C=10,
                                                 max_iter=10000,
                                                 verbose=0))

clf.fit(X_train, Y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=10, max_iter=10000,
                                                 random_state=0))

# Evaluation of Logistic regression multilabels (one-vs-rest)

In [18]:
evaluate_clf(clf, 'results/log_reg_bow.csv', 'Regression logistique bow')

Nombre de tags présents : 56448
Nombre de tags prédits : 44409
---------------------------------------------------------
                    precision    recall  f1-score   support

              .net       0.56      0.25      0.35      2619
    actionscript-3       0.40      0.34      0.37       303
              ajax       0.43      0.39      0.41       470
         algorithm       0.46      0.41      0.44       333
           android       0.84      0.81      0.82       438
            apache       0.50      0.51      0.51       218
       apache-flex       0.74      0.64      0.69       346
            arrays       0.38      0.33      0.35       294
           asp.net       0.74      0.52      0.61      2253
       asp.net-mvc       0.71      0.69      0.70       811
              bash       0.64      0.62      0.63       144
                 c       0.29      0.21      0.24       802
                c#       0.54      0.24      0.34      5163
               c++       0.56      0.3

# SVM multilabels (one vs rest)

Afin de determiner les meilleurs paramètres du modèle, nous allons utiliser `GridSearchCV` du module `model_selection` de `scikit-learn`. Nous appliquerons cela à un échantillon de 15000 posts afin de reduire le temps d'exécution.

In [19]:
# classifier
svm_clf = OneVsRestClassifier(LinearSVC(random_state=0,
                                        class_weight='balanced',
                                        max_iter=10000,
                                        verbose=0))
# hyperparameters
param_grid = [{'estimator__C': [0.1, 0.5, 1., 5, 10]}]

# cross-validation
clf = GridSearchCV(svm_clf,
                   param_grid,
                   scoring='f1_macro',
                   n_jobs=5,
                   verbose=0,
                   return_train_score=True)

clf.fit(X_train_s, Y_train_s)

print('Meilleurs hyper-paramètres :\n')
print(clf.best_params_)

Meilleurs hyper-paramètres :

{'estimator__C': 0.1}


Une fois le meilleur paramètre determiner, nous appliquerons la classification à toute la base en considérant le meilleur paramètre déterminé précédemment.

In [20]:
# train classifier
clf = OneVsRestClassifier(LinearSVC(random_state=0,
                                    class_weight='balanced',
                                    C=0.1,
                                    max_iter=10000,
                                    verbose=0))

clf.fit(X_train, Y_train)

OneVsRestClassifier(estimator=LinearSVC(C=0.1, class_weight='balanced',
                                        max_iter=10000, random_state=0))

# Evaluation of SVM multilabels (one-vs-rest)

In [21]:
evaluate_clf(clf, 'results/svm_bow.csv', 'SVM bow')

Nombre de tags présents : 56448
Nombre de tags prédits : 126004
---------------------------------------------------------
                    precision    recall  f1-score   support

              .net       0.24      0.69      0.35      2619
    actionscript-3       0.22      0.59      0.33       303
              ajax       0.23      0.64      0.33       470
         algorithm       0.29      0.56      0.38       333
           android       0.62      0.84      0.71       438
            apache       0.37      0.68      0.48       218
       apache-flex       0.50      0.76      0.60       346
            arrays       0.21      0.57      0.31       294
           asp.net       0.41      0.77      0.53      2253
       asp.net-mvc       0.44      0.81      0.57       811
              bash       0.37      0.72      0.49       144
                 c       0.15      0.67      0.25       802
                c#       0.34      0.77      0.47      5163
               c++       0.25      0.

# Random forest multilabels (one vs rest) 

In [22]:
# classifier
randf = OneVsRestClassifier(RandomForestClassifier(random_state=0,
                                                   n_estimators=50,
                                                   criterion='gini'))

# # hyperparameters
param_grid = {'estimator__max_depth':[5, 10], #10
              'estimator__max_features':[0.25, 0.5, 0.75], 
              'estimator__min_samples_leaf':[1, 3],}


# # # cross-validation
clf = GridSearchCV(randf,
                   param_grid,
                   scoring='f1_weighted',
                   n_jobs=5,
                   verbose = 1,
                   return_train_score=True)


clf.fit(X_train_s, Y_train_s)

print('Meilleurs hyper-paramètres :\n')
print(clf.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed: 21.0min
[Parallel(n_jobs=5)]: Done  60 out of  60 | elapsed: 42.5min finished


Meilleurs hyper-paramètres :

{'estimator__max_depth': 10, 'estimator__max_features': 0.75, 'estimator__min_samples_leaf': 3}


In [23]:
clf = OneVsRestClassifier(RandomForestClassifier(random_state=0,
                                                 max_depth=10,  # 10
                                                 min_samples_leaf=3, # 3
                                                 max_features=0.75, # 0.75
                                                 n_estimators=50,
                                                 criterion='gini'))

clf.fit(X_train, Y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=10,
                                                     max_features=0.75,
                                                     min_samples_leaf=3,
                                                     n_estimators=50,
                                                     random_state=0))

# Evaluation of Random forest multilabels (one-vs-rest)

In [24]:
evaluate_clf(clf, 'results/rand_forest_bow.csv', 'Random forest bow')

Nombre de tags présents : 56448
Nombre de tags prédits : 33448
---------------------------------------------------------
                    precision    recall  f1-score   support

              .net       0.66      0.34      0.44      2619
    actionscript-3       0.72      0.34      0.47       303
              ajax       0.66      0.46      0.54       470
         algorithm       0.64      0.32      0.42       333
           android       0.97      0.78      0.87       438
            apache       0.67      0.55      0.60       218
       apache-flex       0.83      0.80      0.82       346
            arrays       0.39      0.10      0.15       294
           asp.net       0.80      0.60      0.69      2253
       asp.net-mvc       0.84      0.80      0.82       811
              bash       0.78      0.70      0.74       144
                 c       0.52      0.05      0.10       802
                c#       0.62      0.07      0.13      5163
               c++       0.74      0.1